In [59]:
import os,gc
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.cross_validation import cross_val_score

from mlens.metrics import make_scorer
from mlens.model_selection import Evaluator

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from copy import deepcopy

user_folder = os.path.expanduser("~")
data_folder = os.path.join(os.path.expanduser("~"), 'E:/git/database/Toxic_Comment')

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_target = pd.read_csv(os.path.join(data_folder, "train.csv"))
submission = pd.read_csv(os.path.join(data_folder, "sample_submission.csv"))
gc.collect()

2631

In [60]:
# 读入原始数据
models = ['lr', 'lstm', 'svm', 'rf','lgb', 'textCNN', 'rank_averaged']
train_preds = {}
test_preds = {}

for model in models:
    # train_preds[model] = pd.read_csv(os.path.join(data_folder, "%s_train_preds.csv" % model))
    test_preds[model] = pd.read_csv(os.path.join(data_folder, "%s_submission.csv" % model))
    
gc.collect()

41

In [88]:
class class_data():
    """按类别组织数据"""
    def __init__(self, train, target=None):
        self.__X = train
        self.__models = list(self.__X.keys())
        self.__class = list(self.__X[self.__models[0]].columns)
        
        if target is not None:
            self.__y = target
        else:
            self.__y = None
            
        self.classes = self.__class
        self.models = self.__models
    
    
        
    def __merge(self):
        data = {}
        for label in self.__class:
            label_data = np.zeros((self.__X[self.__models[0]].shape[0], len(self.__models)))
            label_data = pd.DataFrame(label_data); label_data.columns = self.__models
            for model in self.__models:
                label_data[model] = self.__X[model][label]
            if self.__y is not None:
                label_value = self.__y[label]
                label_result = {'X': label_data, 'y': label_value}
            else:
                label_result = {'X': label_data}
                
            data[label] = label_result
        return(data)
    
    @property
    def data(self):
        return(self.__merge())
    
    def __scoring(self, class_X, class_y):
        roc_score = []
        for model in self.__models:
            roc_score.append(roc_auc_score(class_y, class_X[model]))
        roc_score = pd.DataFrame(roc_score)
        roc_score = roc_score.T; roc_score.columns = models
        return(roc_score)
        
    @property
    def roc_score(self):
        data = self.data
        score = {}
        for label in self.__class:
            score[label] = self.__scoring(data[label]['X'], data[label]['y'])
        return (score)
            

In [89]:
def select_score(array):
    data = array
    data_min = min(data)
    data_max = max(data)
    length = len(array)
    
    flag = (data < 0.5).sum()
    
    if flag > 0.5*lengtest_preds['lgb']th:
        score = data_min
    elif flag < 0.5*length:
        score = data_max
    else:
        score = data.mean()
    
    return score

SyntaxError: invalid syntax (<ipython-input-89-bef54376aad5>, line 9)

In [98]:
#train_data = class_data(train_preds, train_target).data
#train_roc_score = class_data(train_preds, train_target).roc_score
test_data = class_data(test_preds).data
del test_data['id']

In [99]:
# base ensembling: average
for label in class_names:
    X_test = test_data[label]["X"]
    result = X_test.apply(np.mean, axis=1)
    submission[label] = result
    
submission.to_csv(os.path.join(data_folder, "ensemble_average_other_submission.csv"), index=False) 

### Toxic

In [20]:
roc_matrix = pd.DataFrame(np.arange(42).reshape((6,7)))
roc_matrix.index = class_names
roc_matrix.columns = models + ['select', 'ave']
# roc_matrix = roc_matrix.T

In [21]:
for label in ['toxic']:
# for label in class_names:
    X_train = train_data[label]['X']
    y_train = train_data[label]['y']
    roc_score = train_roc_score[label]
    #select = X_train.apply(select_score, axis=1)
    #ave = X_train.apply(np.mean, axis=1)
    
    #roc_score['select'] = roc_auc_score(y_train, select)
    #roc_score['ave'] = roc_auc_score(y_train, ave)
    
    #roc_matrix.loc[label,] = roc_score.iloc[0,].tolist()

In [37]:
clf = LogisticRegression()
outFilename = os.path.join(data_folder, "ensemble_lr_submission.csv")
for label in class_names:
    X_train = train_data[label]['X']
    y_train = train_data[label]['y']
    X_test = test_data[label]['X']
                           
    clf.fit(X_train, y_train)
    test_preds = clf.predict_proba(X_test)[:,1]
                           
    submission[label] = test_preds

submission.to_csv(os.path.join(data_folder, "ensemble_lr_submission.csv"), index=False)

In [25]:
# roc_scorer = make_scorer(roc_auc_score, greater_is_better=True)
pars = {
    # "max_features": [50, 200],
    "n_estimators": [20,100],
    # "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2, 4]
}


clf = LogisticRegression()
roc_score = cross_val_score(clf, X_train, y_train, scoring='roc_auc')
# grid = GridSearchCV(clf, pars, scoring='roc_auc', n_jobs=1, cv=5)
# grid.fit(train_tfidf, train_df["toxic"],)
# print("Accuracy: {0:.4f}%".format(grid.best_score_ * 100))
gc.collect()
print(clf)
print(roc_score)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.99985959 0.99983043 0.99981156]
